In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
os.getcwd()
os.chdir('/content/drive/My Drive/deep learning1')

In [ ]:
apple_training_complete = pd.read_csv('apple_training.csv')

In [ ]:
apple_training_complete.shape

(1260, 7)

In [ ]:
apple_training_processed = apple_training_complete.iloc[:, 1:2].values  


In [ ]:
apple_training_processed.shape

(1260, 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler  
scaler = MinMaxScaler(feature_range = (0, 1))

apple_training_scaled = scaler.fit_transform(apple_training_processed)  

In [ ]:
apple_training_scaled.shape

(1260, 1)

In [ ]:
apple_training_processed #data before scaling

array([[ 72.932854],
       [ 79.117142],
       [ 78.26857 ],
       ...,
       [170.100006],
       [171.      ],
       [170.520004]])

In [ ]:
apple_training_scaled


array([[0.14628787],
       [0.19795893],
       [0.19086893],
       ...,
       [0.95814041],
       [0.96566005],
       [0.96164958]])

In [ ]:
features_set = []  
labels = []  
for i in range(60, 1260):  
    features_set.append(apple_training_scaled[i-60:i,0])
    labels.append(apple_training_scaled[i, 0])

In [ ]:
len(features_set)

1200

In [ ]:
features_set, labels = np.array(features_set), np.array(labels)  


In [ ]:
features_set.shape

(1200, 60)

In [ ]:
labels.shape

(1200,)

In [ ]:
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))  


In [ ]:
features_set.shape

(1200, 60, 1)

In [ ]:
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  
from keras.layers import Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))  


In [ ]:
model.add(Dropout(0.2))  


In [ ]:
model.add(LSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))

model.add(LSTM(units=50))  
model.add(Dropout(0.2))  

In [ ]:
model.add(Dense(units = 1))  


In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')  


In [ ]:
model.fit(features_set, labels, epochs = 100, batch_size = 32) 


38/38 [==============================] - 8s 212ms/step - loss: 0.0036
Epoch 11/100
18/38 [=============>................] - ETA: 4s - loss: 0.0037

# Testing the LSTM

In [ ]:
apple_testing_complete = pd.read_csv('apple_testing.csv')  
apple_testing_processed = apple_testing_complete.iloc[:, 1:2].values  

In [ ]:
apple_testing_processed.shape

Converting test data into right format
For each day of January 2018, we want our feature set to contain the opening stock prices for the previous 60 days. For the 1st of January, we need the stock prices for the previous 60 days. To do so, we need to concatenate our training data and test data before preprocessing

In [ ]:
apple_total = pd.concat((apple_training_complete['Open'], apple_testing_complete['Open']), axis=0)


In [ ]:
apple_total.shape

Now let's prepare our test inputs. The input for each day should contain the opening stock prices for the previous 60 days. That means we need opening stock prices for the 20 test days for the month of January 2018 and the 60 stock prices from the last 60 days for the training set.

In [ ]:
test_inputs = apple_total[len(apple_total) - len(apple_testing_complete) - 60:].values 

In [ ]:
test_inputs.shape

Scale our test data as we did in train data

In [ ]:
test_inputs = test_inputs.reshape(-1,1) 

In [ ]:
test_inputs.shape

In [ ]:
 
test_inputs = scaler.transform(test_inputs)  

In [ ]:
test_inputs[0]

In [ ]:
test_features = []  
for i in range(60, 80):  
    test_features.append(test_inputs[i-60:i, 0])


In [ ]:
test_features = np.array(test_features) 

In [ ]:
test_features.shape

In [ ]:
 
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))  

In [ ]:
test_features.shape

In [ ]:
predictions = model.predict(test_features)  

In [ ]:
predictions

Since we scaled our data, the predictions made by the LSTM are also scaled. We need to reverse the scaled prediction back to their actual values. To do so, we can use the ìnverse_transform method of the scaler object we created during training.

In [ ]:
predictions = scaler.inverse_transform(predictions)

In [ ]:
predictions

In [ ]:
plt.figure(figsize=(10,6))  
plt.plot(apple_testing_processed, color='blue', label='Actual Apple Stock Price')  
plt.plot(predictions , color='red', label='Predicted Apple Stock Price')  
plt.title('Apple Stock Price Prediction')  
plt.xlabel('Date')  
plt.ylabel('Apple Stock Price')  
plt.legend()  
plt.show()  

Conclusion
A long short-term memory network (LSTM) is one of the most commonly used neural networks for time series analysis. The ability of LSTM to remember previous information makes it ideal for such tasks. 
  we can use LSTM for the Apple stock price prediction.